In [1]:
import sys
# Aggiungi il percorso della cartella che contiene utils.py a sys.path
sys.path.append('../utils')

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid')

from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2
from NN import NeuralNetwork
from joblib import dump

In [3]:
TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'neural_network')
MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [4]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') # casting
df_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [5]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target_x,target_y,target_z
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [6]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

# TODO split potenzialmente variabile?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model

In [7]:
def build_model(architecture, activation, learning_rate, momentum, nesterov, weight_decay, input_dimension,
                output_dimension, dropout_input_rate, dropout_hidden_rate):

    nn = NeuralNetwork(
        architecture=architecture,
        activation=activation,
        learning_rate=learning_rate,
        momentum=momentum,
        use_nesterov=nesterov,
        weight_decay=weight_decay,
        input_dimension=input_dimension,
        output_dimension=output_dimension,
        dropout_input_rate=dropout_input_rate,
        dropout_hidden_rate=dropout_hidden_rate
    )

    return nn.build_model()

In [8]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', KerasRegressor(build_model, epochs=100, batch_size=32, verbose=0))
])

In [9]:
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV

search_spaces = {
    'neuralnetwork__model__architecture': Categorical([(64, 64), (128, 64), (128, 128)]),
    'neuralnetwork__model__activation': Categorical(['relu']),
    'neuralnetwork__model__learning_rate': Real(0.0001, 0.01, prior='log-uniform'),
    'neuralnetwork__model__momentum': Real(0.5, 0.9, prior='uniform'),
    'neuralnetwork__model__nesterov': Categorical([True, False]),
    'neuralnetwork__model__weight_decay': Real(1e-6, 1e-4, prior='log-uniform'),
    'neuralnetwork__model__input_dimension': Categorical([10]),
    'neuralnetwork__model__output_dimension': Categorical([3]),
    'neuralnetwork__model__dropout_input_rate': Real(0.0, 0.5, prior='uniform'),
    'neuralnetwork__model__dropout_hidden_rate': Real(0.0, 0.5, prior='uniform')
}




In [10]:
from sklearn.metrics import make_scorer

mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer}

In [11]:
bayes_search = BayesSearchCV(
    pipeline,
    search_spaces=search_spaces,
    cv=5,
    scoring=scoring,
    refit='MEE',
    n_jobs=-1,
    verbose= 2,
    n_iter=30,  # Numero di iterazioni di ottimizzazione
)
np.int = int


bayes_search.fit(X_train, y_train)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
best_params = bayes_search.best_params_
best_index = bayes_search.best_index_

mee = bayes_search.best_score_
r2 = bayes_search.cv_results_['mean_test_R2'][best_index]
rmse = bayes_search.cv_results_['mean_test_RMSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))

In [ ]:
final_model = bayes_search.best_estimator_
final_model

In [ ]:
final_model.named_steps['neuralnetwork'].model

# Test of the model

In [ ]:
y_pred = final_model.predict(X_test)

mee = mean_euclidean_error(y_test, y_pred)
r2 = multidim_r2(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

print('MEE:', mee)
print('R2:', r2)
print('RMSE:', rmse)

# Save model

In [ ]:
model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)